In [301]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import spacy

import re
import os

In [3]:
!pip uninstall neuralcoref

In [1]:
DOCUMENT = """
The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios 
and published by Bethesda Softworks. It is the fifth main installment in The Elder Scrolls series, 
following The Elder Scrolls IV: Oblivion.
The game's main story revolves around the player character's quest to defeat Alduin the World-Eater, 
a dragon who is prophesied to destroy the world. The game is set 200 years after the events of Oblivion 
and takes place in the fictional province of Skyrim. Over the course of the game, the player completes 
quests and develops the character by improving skills. The game continues the open-world tradition of 
its predecessors by allowing the player to travel anywhere in the game world at any time, and to ignore 
or postpone the main storyline indefinitely.
The team opted for a unique and more diverse open world than Oblivion's Imperial Province of Cyrodiil, 
which game director and executive producer Todd Howard considered less interesting by comparison. 
The game was released to critical acclaim, with reviewers particularly mentioning the character advancement 
and setting, and is considered to be one of the greatest video games of all time.


The Elder Scrolls V: Skyrim is an action role-playing game, playable from either a first or 
third-person perspective. The player may freely roam over the land of Skyrim which is an open world 
environment consisting of wilderness expanses, dungeons, cities, towns, fortresses, and villages. 
Players may navigate the game world more quickly by riding horses or by utilizing a fast-travel system 
which allows them to warp to previously discovered locations. The game's main quest can be completed or 
ignored at the player's preference after the first stage of the quest is finished. However, some quests 
rely on the main storyline being at least partially completed. Non-player characters (NPCs) populate the 
world and can be interacted with in a number of ways: the player may engage them in conversation, 
marry an eligible NPC, kill them or engage in a nonlethal "brawl". The player may 
choose to join factions which are organized groups of NPCs — for example, the Dark Brotherhood, a band 
of assassins. Each of the factions has an associated quest path to progress through. Each city and town 
in the game world has jobs that the player can engage in, such as farming.

Players have the option to develop their character. At the beginning of the game, players create 
their character by selecting their sex and choosing between one of several races including humans, 
orcs, elves, and anthropomorphic cat or lizard-like creatures and then customizing their character's 
appearance. Over the course of the game, players improve their character's skills which are numerical 
representations of their ability in certain areas. There are eighteen skills divided evenly among the 
three schools of combat, magic, and stealth. When players have trained skills enough to meet the 
required experience, their character levels up. Health is depleted primarily when the player 
takes damage and the loss of all health results in death. Magicka is depleted by the use of spells, 
certain poisons and by being struck by lightning-based attacks. Stamina determines the player's 
effectiveness in combat and is depleted by sprinting, performing heavy "power attacks" 
and being struck by frost-based attacks. Skyrim is the first entry in The Elder Scrolls to 
include dragons in the game's wilderness. Like other creatures, dragons are generated randomly in 
the world and will engage in combat with NPCs, creatures and the player. Some dragons may attack 
cities and towns when in their proximity. The player character can absorb the souls of dragons 
in order to use powerful spells called "dragon shouts" or "Thu'um". A regeneration 
period limits the player's use of shouts in gameplay.

Skyrim is set around 200 years after the events of The Elder Scrolls IV: Oblivion, although it is 
not a direct sequel. The game takes place in Skyrim, a province of the Empire on the continent of 
Tamriel, amid a civil war between two factions: the Stormcloaks, led by Ulfric Stormcloak, and the 
Imperial Legion, led by General Tullius. The player character is a Dragonborn, a mortal born with 
the soul and power of a dragon. Alduin, a large black dragon who returns to the land after being 
lost in time, serves as the game's primary antagonist. Alduin is the first dragon created by Akatosh, 
one of the series' gods, and is prophesied to destroy and consume the world.
"""

In [ ]:
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()

In [1]:
# 1. Add conjunct direct object (DONE)
# 2. Add xcomp after verb ==> Add preposition or adverb or adjective (the pattern similar like passive sentence case)
# 3. Store all possible verb conjunct and its index. It help re-looping unnecessary situation.
# 4. Is pre-positional adverb before verb necessary? (DONE) ==> Conjunct adverb is not needed.
# 5. I need to go shopping, but I have no time. ==> Check neglection in the rules (DONE)

def subject_active_rules(token):
    abilities = []
    # Go to its head
    head = token.head

    # Get neglect; If there is no neglect, return empty text.
    neg = get_neglect(head)

    # If head is Verb.
    if head.pos_ == 'VERB':

        # Get all conjunct (except adjective with dependency acomp)
        if neg:
            verb_conjunct = extract_conj(head)
        else:
            # If neglect do not come at first, then check neglection in front each conjunct.
            verb_conjunct = extract_conj(head, neglect=True)

        ###============ GET ALL TOKENS ================###
        # Get aux
        aux = get_token_dep(head, dep='aux')
            
        # Get pre and post adverb after verb
        advmod_right_main = get_token_dep_right(head, dep=['advmod', 'npadvmod'])
        post_adv = []
        if advmod_right_main:
            # Get pre and post adverb after post main verb
            pre_advmod_right_main, post_advmod_right_main = extract_adv(advmod_right_main)
            pre_advmod_right_main = ' '.join(pre_advmod_right_main)
            post_advmod_right_main = ' '.join(post_advmod_right_main)

            post_adv = [pre_advmod_right_main, advmod_right_main.text, post_advmod_right_main]
        post_adv = (' '.join(post_adv)).strip()

        advmod_left_main = get_token_dep_left(head, dep=['advmod', 'npadvmod'])
        pre_adv = []
        if advmod_left_main:
            # Get pre and post adverb after pre main verb
            pre_advmod_left_main, post_advmod_left_main = extract_adv(advmod_left_main)
            pre_advmod_left_main = ' '.join(pre_advmod_left_main)
            post_advmod_left_main = ' '.join(post_advmod_left_main)
                
            pre_adv = [pre_advmod_left_main, advmod_left_main.text, post_advmod_left_main]
        pre_adv = (' '.join(pre_adv)).strip()
            

        # Get Preposition after verb
        prep_after_verb = crawling_after_token_prep_phrase(head)

        # Get direct object verb
        obj = get_token_dep(head, dep='dobj')
            
        # Get intransitive rate score
        int_rate = map_verb_intrans.get(head.text) or map_verb_intrans.get(head.lemma_)
        # If the verb is not in the mapper ( we assume it is transitive verb )
        if not int_rate:
            int_rate = 0

        # Get xcomp token
        xcomp = get_token_dep(head, dep='xcomp')
        # Initalize object and advmod of xcomp.
        obj_xcomp = None
        advmod_xcomp = None
        if xcomp:
            # Get the aux, adv, and obj of xcomp tokens.
            aux_xcomp = get_token_dep(xcomp, dep='aux')
            obj_xcomp = get_token_dep(xcomp, dep=['pobj', 'dobj'])
            advmod_xcomp = get_token_dep_right(xcomp, dep='advmod')
            # If adv modifier of xcomp exist
            if advmod_xcomp:
                # Get pre and post adverb of main adverb modifier xcomp.
                pre_advmod_xcomp, post_advmod_xcomp = extract_adv(advmod_xcomp)
                pre_advmod_xcomp = ' '.join(pre_advmod_xcomp)
                post_advmod_xcomp = ' '.join(post_advmod_xcomp)



        ###===================== CONDITION =====================###


        if (advmod_right_main) or (prep_after_verb) or (obj) or (xcomp):
            # If adverb after verb exist ==> Subject + aux (optional) + not (optional) + adv (optional) + verb + adv + preposition phrase (optional)
            if advmod_right_main:
                # Concatenate components into: adv (optional) + verb
                ability = cross_product_str(pre_adv, head.text)
                # Concatenate components into: not (optional) + adv (optional) + verb
                ability = cross_product_str(neg, ability)
                # Concatenate components into: not (optional) + adv (optional) + verb + adv
                ability = cross_product_str(ability, post_adv)
        
                # If aux exist, add the text into ability_text
                if aux:
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adv
                    ability = cross_product_str(aux.text, ability)

                # If preposition phrase after adverb exist.
                prep_after_advmod = crawling_after_token_prep_phrase(advmod_right_main)
                if prep_after_advmod:
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adv + preposition phrase (optional)
                    ability = cross_product_str(ability, prep_after_advmod)
                    
                # EXPECTED PATTERN: Subject + aux (optional) + not (optional) + adv (optional) + verb + adv + prepositional phrase (optional)
                abilities += ability

            # If prep after verb ==> Subject + aux (optional) + not (optional) + adv (optional) + verb + preposition phrase
            if prep_after_verb:
                # Concatenate components into: adv (optional) + verb
                ability = cross_product_str(pre_adv, head.text)
                # Concatenate components into: not (optional) + adv (optional) + verb
                ability = cross_product_str(neg, ability)
                # If auxiliary token exist
                if aux:
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                    ability = cross_product_str(aux.text, ability)
    
                # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + preposition phrase
                ability = cross_product_str(ability, prep_after_verb)
    
                # EXPECTED PATTERN: Subject + aux (optional) + not (optional) + adv (optional) + verb + preposition phrase
                abilities += ability

            # If direct object exist ==> Subject + aux (optional) + not (optional) + adv (optional) + verb + adj (optional) + Direct object
            if obj:
                # If the neglection do not appear as children's verb
                if not neg:
                    neg = get_neglect(obj)
                # Get pre adjective modifier direct object and convert in into string
                pre_adj = ' '.join(extract_pre_adj(obj))
                # Concatenate components into: adv (optional) + verb
                ability = cross_product_str(pre_adv, head.text)
                # Concatenate components into: not (optional) + adv (optional) + verb
                ability = cross_product_str(neg, ability)
                # If auxiliary token exist
                if aux:
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                    ability = cross_product_str(aux.text, ability)
                # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adj (optional)
                ability = cross_product_str(ability, pre_adj)
                # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adj (optional) + Direct object
                ability = cross_product_str(ability, obj.text)
                    
                # EXPECTED PATTERN: Subject + aux (optional) + not (optional) + adv (optional) + verb + adj (optional) + Direct object
                abilities += ability

                # If the object has conjunct
                obj_conjunct = extract_conj(obj)
                if len(obj_conjunct) > 0:
                    # Concatenate components into: Verb (Have) + Conjunct
                    ability = cross_product_str(head.text, obj_conjunct)
                    # Concatenate components into: not (optional) + Verb (Have) + Conjunct
                    ability = cross_product_str(neg, ability)
    
                    # Add the ability into abilities
                    abilities += ability
                    # EXPECTED PATTERN: Subject + not (optional) + Verb (Have) + Direct object
                    # Note: Since normaly, If direct object is noun/propn/pron the conjuncts are noun/propn/pron too.
                    #        This rule follow this concept. In somehow, the conjunct could be adjective or another verb.

            if xcomp:
                print(xcomp)
                # # Concatenate components into: adv (optional) + verb
                # ability = cross_product_str(pre_adv, head.text)
                # # Concatenate components into: not (optional) + adv (optional) + verb
                # ability = cross_product_str(neg, ability)
                # # If auxiliary token exist
                # if aux:
                #     # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                #     ability = cross_product_str(aux.text, ability)
    
                # # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp
                # ability = cross_product_str(ability, aux_xcomp.text)
                # # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp
                # ability = cross_product_str(ability, xcomp.text)
        
                # # Store ability: If object xcomp exist
                # if obj_xcomp:
                #     # Get the pre adjective of object
                #     pre_adj = ' '.join(extract_pre_adj(obj_xcomp))
                #     # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + adj (optional)
                #     ability = cross_product_str(ability, pre_adj)
                #     # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + adj (optional) + object
                #     ability = cross_product_str(ability, obj_xcomp.text)
                #     # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp  + adj (optional) + object
                #     abilities += ability
                            
                #     # Get all object conjuncts
                #     if neg:
                #         obj_xcomp_conj = extract_conj(obj_xcomp)
                #     else:
                #         # If neglect do not come at first, then check neglection in front each conjunct.
                #         obj_xcomp_conj = extract_conj(obj_xcomp, neglect=True)
                    
                #     # If object conjuncts exist
                #     if len(obj_xcomp_conj) > 0:
                #         # Concatenate components into: adv (optional) + verb
                #         ability = cross_product_str(pre_adv, head.text)
                #         # Concatenate components into: not (optional) + adv (optional) + verb
                #         ability = cross_product_str(neg, ability)
                #         # If auxiliary token exist
                #         if aux:
                #             # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                #             ability = cross_product_str(aux.text, ability)
                        
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp
                #         ability = cross_product_str(ability, aux_xcomp.text)
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp
                #         ability = cross_product_str(ability, xcomp.text)
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + object
                #         ability = cross_product_str(ability, obj_xcomp_conj)
                #         # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + object
                #         abilities += ability
                
                # # Store ability: If advmod xcomp exist
                # elif advmod_xcomp:
                #     # If pre advmod xcomp exist
                #     if pre_advmod_xcomp:
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + pre-advmod (optional)
                #         ability = cross_product_str(ability, pre_advmod_xcomp)
                        
                #     # Concatenate components:aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + pre-advmod (optional) + advmod
                #     ability = cross_product_str(ability, advmod_xcomp.text)
                #     # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + pre-advmod (optional) + advmod
                #     abilities += ability  
        
                #     # Get all advmod conjuncts
                #     if neg:
                #         advmod_xcomp_conj = extract_conj(advmod_xcomp)
                #     else:
                #         # If neglect do not come at first, then check neglection in front each conjunct.
                #         advmod_xcomp_conj = extract_conj(advmod_xcomp, neglect=True)
                #     # If advmod conjuncts exist
                #     if len(advmod_xcomp_conj) > 0:
                #         # Concatenate components into: adv (optional) + verb
                #         ability = cross_product_str(pre_adv, head.text)
                #         # Concatenate components into: not (optional) + adv (optional) + verb
                #         ability = cross_product_str(neg, ability)
                #         # If auxiliary token exist
                #         if aux:
                #             # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                #             ability = cross_product_str(aux.text, ability)
                            
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp
                #         ability = cross_product_str(ability, aux_xcomp.text)
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp
                #         ability = cross_product_str(ability, xcomp.text)
                #         # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + advmod
                #         ability = cross_product_str(ability, advmod_xcomp_conj)
                #         # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb + aux-xcomp + xcomp + advmod
                #         abilities += ability


        else:
            # If do not contain any of that, but intransitive verb ==> Subject + aux (optional) + not (optional) + adv (optional) + verb
            if int_rate > 0.5:
                # Concatenate components into: adv (optional) + verb
                ability = cross_product_str(pre_adv, head.text)
                # Concatenate components into: not (optional) + adv (optional) + verb
                ability = cross_product_str(neg, ability)
        
                # If aux exist, add the text into ability_text
                if aux:
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                    ability = cross_product_str(aux.text, ability)

                # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb
                abilities += ability

    # If head is aux
    elif head.pos_ == 'AUX':
        # Get the token
        # NOTE: if 'AUX' is root, only have one adjective with dependency acomp.
        adj_token = get_token_dep(head, dep='acomp')
        noun_token = get_token_pos_right(head, pos=['NOUN', 'PROPN'])

        if adj_token:
            # Get Preposition after adjective
            prep_after_adj = ' '.join(crawling_after_token_prep_phrase(adj_token))

            # Concatenate components into: aux + not (optional)
            ability = cross_product_str(head.text, neg)
            # Concatenate components into: aux + not (optional) + adj
            ability = cross_product_str(ability, adj_token.text)
            # Concatenate components into: aux + not (optional) + adj + preposition phrase (optional)
            ability = cross_product_str(ability, prep_after_adj)

            # EXPECTED PATTERN: Subject + aux + not (optional) + adj
            abilities += ability

            # Get all conjunct (except adjective with dependency acomp)
            if neg:
                adj_conjunct = extract_conj(adj_token)
            else:
                # If neglect do not come at first, then check neglection in front each conjunct.
                adj_conjunct = extract_conj(adj_token, neglect=True)

            if len(adj_conjunct) > 0:
                # Concatenate components into: aux + not (optional)
                ability = cross_product_str(head.text, neg)
                # Concatenate components into: aux + not (optional) + adj
                ability = cross_product_str(ability, adj_conjunct)
                # EXPECTED PATTERN: Subject + aux + not (optional) + adj
                abilities += ability

        elif noun_token:
            # Get Pre-modifier adjective
            pre_adj_noun = ' '.join(extract_pre_adj(noun_token))
            # Concatenate components into: aux + not (optional)
            ability = cross_product_str(head.text, neg)
            # Concatenate components into: aux + not (optional) + pre-modifier adjective (optional)
            ability = cross_product_str(ability, pre_adj_noun)
            # Concatenate components into: aux + not (optional) + pre-modifier adjective (optional) + noun
            ability = cross_product_str(ability, noun_token.text)

            # EXPECTED PATTERN: Subject + aux + not (optional) + pre-modifier adjective (optional) + noun
            abilities += ability

            # Get all conjunct
            if neg:
                noun_conjunct = extract_conj(noun_token)
            else:
                # If neglect do not come at first, then check neglection in front each conjunct.
                noun_conjunct = extract_conj(noun_token, neglect=True)

            if len(noun_conjunct) > 0:
                # Concatenate components into: aux + not (optional)
                ability = cross_product_str(head.text, neg)
                # Concatenate components into: aux + not (optional) + noun
                ability = cross_product_str(ability, noun_conjunct)
                # EXPECTED PATTERN: Subject + aux + not (optional) + noun
                abilities += ability
    return abilities

In [ ]:
def base_sentence_comp(comp, **kwargs):
    # Generate base sentence: aux (optional) + not (optional) + adv (optional) + verb
    ability = base_sentence(main_aux=kwargs['main_aux'],
                            main_verb=kwargs['main_verb'],
                            main_pre_adv=kwargs['main_pre_adv'], 
                            neg=kwargs['neg'])
    if kwargs['aux_comp']:
        # Concatenate components: aux (optional) + not (optional) + adv (optional) 
        #                           + verb + aux-comp (optional)
        ability = cross_product_str(ability, kwargs['aux_comp'].text)
    # Concatenate components: aux (optional) + not (optional) + adv (optional) 
    #                           + verb + aux-comp (optional) + compliment
    ability = cross_product_str(ability, kwargs['comp'].text)
    
    return ability

In [ ]:
def ability_comp_advmod(comp, advmod, base=None, **kwargs):
    # Get pre and post adverb of main adverb modifier compliment.
    pre_advmod, post_advmod = extract_adv(advmod)
    pre_advmod = ' '.join(pre_advmod)
    post_advmod = ' '.join(post_advmod)

    # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb 
    #                           + aux-aux-comp (optional) + compliment
    ability = base_sentence_comp(comp, **kwargs)
    # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb 
    #                           + aux-comp (optional) + compliment + pre-adverb
    ability = cross_product_str(ability, pre_advmod)
    # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb 
    #                           + aux-comp (optional) + compliment + pre-adverb mod + advmod
    ability = cross_product_str(ability, advmod.text)
    # Concatenate components: aux (optional) + not (optional) + adv (optional) + verb 
    #                           + aux-comp (optional) + compliment + pre-adverb modifier
    #                           + advmod + post-adverb modifier
    ability = cross_product_str(ability, post_advmod)

    return ability

In [9]:
# 1. ACL
# 2. Preprosition phrase
# 3. Conjunction



In [ ]:
def ability_adnominal_clause(acl, **kwargs):
    # Define local variable
    result = []
    aux_acl, obj_acl, advmod_acl, prep_acl, acomp_acl = get_components_comp(acl).values()

    # Add auxiliary clause into kwargs
    kwargs['aux_comp'] = aux_acl
    if (obj_acl) or (advmod_acl) or (prep_acl) or (acomp_acl):
        # If direct object exist
        if obj_acl:                    
            ability = ability_comp_dobj(comp=acl, obj=obj_acl, neglect=False, **kwargs)
            result += ability
            
        # If advmod exist
        if advmod_acl:
            ability = ability_comp_advmod(comp=acl, advmod=advmod_acl, **kwargs)
            result += ability
    
        # If preposition after acl appears
        if prep_acl:
            # ability = ability_comp_prep(comp=xcomp, prep=prep_xcomp, **components)
            ability = ability_prep(prep_acl, comp=acl, **kwargs)
            result += ability
        
        # If acomp after acl appears
        if acomp_acl:
            ability = ability_comp_acomp(comp=acl, acomp=acomp_acl, **kwargs)
            result += ability
    return result
    

In [ ]:
                    if obj_xcomp:
                        print("FOUR")
                        # ability = ability_comp_dobj(comp=xcomp, obj=obj_xcomp, neglect=False, **components)
                        ability = ability_dobj(obj_xcomp, comp=xcomp, **components)
                        abilities += ability
            
                    # If advmod exist
                    if advmod_xcomp:
                        print("FIVE")
                        # ability = ability_comp_advmod(comp=xcomp, advmod=advmod_xcomp, **components)
                        ability = ability_advmod(advmod_xcomp, comp=xcomp, **components)
                        abilities += ability
    
                    # If preposition after xcomp appears
                    if prep_xcomp:
                        print("SIX")
                        # ability = ability_comp_prep(comp=xcomp, prep=prep_xcomp, **components)
                        ability = ability_prep(prep_xcomp, comp=xcomp, **components)
                        abilities += ability
        
                    # If acomp after xcomp appears
                    if acomp_xcomp:
                        print("SEVEN")
                        # ability = ability_comp_acomp(comp=xcomp, acomp=acomp_xcomp, **components)
                        # Concatenate components: aux (optional) + not (optional) + adv (optional) 
                        #                           + verb + aux-comp (optional) + compliment
                        ability = base_sentence_comp(xcomp, **components)
                        # Concatenate components: aux (optional) + not (optional) + adv (optional) 
                        #                           + verb + aux-comp (optional) + compliment
                        #                           + adjectival complement
                        ability = cross_product_str(ability, acomp_xcomp.text)
                        abilities += ability


In [7]:
a, b, c = {'a': 1, 'b': 2}.values()

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
def ability_verb(main_verb):
    result = []
    ###============ DEFINE VARIABLES ================###
    aux, obj, pre_adv, advmod_right, prep, acomp = get_components_verb(head).values()
    # Get intransitive rate score
    int_rate = map_verb_intrans.get(main_verb.text) or map_verb_intrans.get(main_verb.lemma_)
    # If the verb is not in the mapper ( we assume it is transitive verb )
    if not int_rate:
        int_rate = 0

    #==================== COMPLIMENT ========================#
    # Get compliment verb
    comp = get_token_dep(main_verb, dep=['xcomp', 'ccomp'])
    if comp and (comp.pos_ != 'VERB' or get_token_dep(comp, dep='auxpass')):
        comp = None
        
    # NOTE: a single verb to directly have both a ccomp and an xcomp dependency simultaneously 
    #         is rare and typically wouldn't occur. If a verb does have two clausal complements, 
    #         each clause would serve a different function or role in the sentence.
    int_rate_comp = map_verb_intrans.get(main_verb.text) or map_verb_intrans.get(main_verb.lemma_)
    if not int_rate_comp:
        int_rate_comp = 0
        
    ###===================== CONDITION =====================###
    components = {'main_aux': aux, 'main_verb': main_verb, 'main_pre_adv': pre_adv,
                  'neg': neg, }
    result += ability_relative_verb(main_verb, **components)
    if comp:
        components['aux_comp'] = get_token_dep(comp, dep='aux')               
        temp = ability_relative_verb(comp, is_comp=True, **components)
        result += temp
        if len(temp) == 0:
            # If do not contain any of that, but intransitive verb ==> Subject + aux (optional) + not (optional) + adv (optional) + verb
            if int_rate_comp > 0.5:
                ability = base_sentence_comp(comp=comp, **components)
                # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb
                result += ability            

    if len(abilities) == 0:
        # If do not contain any of that, but intransitive verb ==> Subject + aux (optional) + not (optional) + adv (optional) + verb
        if int_rate > 0.5:
            ability = base_sentence(**components)
            # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb
            result += ability

    return result

In [10]:
### HOBBS ALGORITHM

import spacy

# Load the spacy model for English
nlp = spacy.load("en_core_web_sm")

def find_np_vp_left(tree):
    """Helper function to find the NP or S immediately dominating the pronoun."""
    for child in tree.lefts:
        if child.dep_ == "np" or child.dep_ == "s":
            return child
    return None

def find_antecedent(sentence, pronoun):
    """Implementation of Hobbs' algorithm using spaCy."""
    doc = nlp(sentence)
    pronoun_token = None

    # Step 1: Find the pronoun in the sentence
    for token in doc:
        if token.text.lower() == pronoun.lower() and token.pos_ == 'PRON':
            pronoun_token = token
            break

    if pronoun_token is None:
        return None
    
    # Step 2: Get the parse tree rooted at the S or NP immediately dominating the pronoun
    current = pronoun_token
    while current.dep_ not in ('ROOT', 's', 'np'):
        current = current.head

    # Step 3: Traverse the parse tree to the left to find an antecedent
    antecedent = find_np_vp_left(current)
    if antecedent:
        return antecedent

    # Step 4: Traverse the higher nodes in the parse tree to find more antecedents
    while current.head != current:
        current = current.head

        # Step 5: Traverse the left siblings of the current node
        antecedent = find_np_vp_left(current)
        if antecedent:
            return antecedent
    
    return None

# Example usage
sentence = "John saw a dog near the park. He said it was very cute."
pronoun = "He"
antecedent = find_antecedent(sentence, pronoun)

if antecedent:
    print(f"The antecedent for '{pronoun}' is '{antecedent.text}'")
else:
    print(f"No antecedent found for '{pronoun}'")


No antecedent found for 'He'


In [ ]:
# 1. Add conjunct direct object (DONE)
# 2. Add xcomp after verb ==> Add preposition or adverb or adjective (the pattern similar like passive sentence case)
# 3. Store all possible verb conjunct and its index. It help re-looping unnecessary situation.
# 4. Is pre-positional adverb before verb necessary? (DONE) ==> Conjunct adverb is not needed.
# 5. I need to go shopping, but I have no time. ==> Check neglection in the rules (DONE)

def subject_active_rules(token):
    abilities = []
    # Go to its head
    head = token.head

    # Get neglect; If there is no neglect, return empty text.
    neg = get_neglect(head)

    # If head is Verb.
    if head.pos_ == 'VERB':

        # Get all conjunct (except adjective with dependency acomp)
        if neg:
            verb_conjunct = extract_conj(head)
        else:
            # If neglect do not come at first, then check neglection in front each conjunct.
            verb_conjunct = extract_conj(head, neglect=True)

        #### ====== PUT FOR LOOP EACT VERB CONJUNCT HERE ========== ###

        # If the verb is posession
        if head.lemma_ == 'have':

            # Get direct object
            obj = get_token_dep(head, dep='dobj')

            # If direct object exist
            if obj:
                # If the neglection do not appear as children's verb
                if not neg:
                    neg = get_neglect(obj)
                # Extract all possible pre-modifier adjectives
                pre_adj_text = ' '.join(extract_pre_adj(obj))

                # Concatenate components into: Adj (optional) + Direct object
                ability = cross_product_str(pre_adj_text, obj.text)
                # Concatenate components into: Verb (Have) + Adj (optional) + Direct object
                ability = cross_product_str(head.lemma_, ability)
                # Concatenate components into: not (optional) + Verb (Have) + Adj (optional) + Direct object
                ability = cross_product_str(neg, ability)

                # Add the ability into abilities
                abilities += ability
                # EXPECTED PATTERN: Subject + not (optional) + Verb (Have) + Adj (optional) + Direct object

                # If the object has conjunct
                obj_conjunct = extract_conj(obj)
                if len(obj_conjunct) > 0:
                    # Concatenate components into: Verb (Have) + Conjunct
                    ability = cross_product_str(head.lemma_, obj_conjunct)
                    # Concatenate components into: not (optional) + Verb (Have) + Conjunct
                    ability = cross_product_str(neg, ability)

                    # Add the ability into abilities
                    abilities += ability
                    # EXPECTED PATTERN: Subject + not (optional) + Verb (Have) + Direct object
                    # Note: Since normaly, If direct object is noun/propn/pron the conjuncts are noun/propn/pron too.
                    #        This rule follow this concept. In somehow, the conjunct could be adjective or another verb.

            # If the verb is posession but do not have direct object (object that the subject posessed) return empty list
            else:
                return []
        else:
            ## ============ GET POSSIBLY COMPONENTS REQUIREMENT =========== ##
            # Get aux
            aux = get_token_dep(head, dep='aux')

            ##### FIX HERE ######
            # Get Pre and post adv
            # pre_adv, post_adv = extract_adv(head)
            # # Convert pre and post adverb into string
            # pre_adv = (' '.join(pre_adv)).strip()
            # post_adv = (' '.join(post_adv)).strip()
            
            # Get pre and post adverb after verb
            advmod_right_main = get_token_dep_right(head, dep=['advmod', 'npadvmod'])
            post_adv = []
            if advmod_right_main:
                # Get pre and post adverb after post main verb
                pre_advmod_right_main, post_advmod_right_main = extract_adv(advmod_right_main)
                pre_advmod_right_main = ' '.join(pre_advmod_right_main)
                post_advmod_right_main = ' '.join(post_advmod_right_main)

                post_adv = [pre_advmod_right_main, advmod_right_main.text, post_advmod_right_main]
            post_adv = (' '.join(post_adv)).strip()

            advmod_left_main = get_token_dep_left(head, dep=['advmod', 'npadvmod'])
            pre_adv = []
            if advmod_left_main:
                # Get pre and post adverb after pre main verb
                pre_advmod_left_main, post_advmod_left_main = extract_adv(advmod_left_main)
                pre_advmod_left_main = ' '.join(pre_advmod_left_main)
                post_advmod_left_main = ' '.join(post_advmod_left_main)
                
                pre_adv = [pre_advmod_left_main, advmod_left_main.text, post_advmod_left_main]
            pre_adv = (' '.join(pre_adv)).strip()
            

            # Get Preposition after verb
            prep_after_verb = crawling_after_token_prep_phrase(head)

            # Get direct object verb
            obj = get_token_dep(head, dep='dobj')
            
            # Get intransitive rate score
            int_rate = map_verb_intrans.get(head.text) or map_verb_intrans.get(head.lemma_)
            # If the verb is not in the mapper ( we assume it is transitive verb )
            if not int_rate:
                int_rate = 0


            ###===================== CONDITION =====================###

            if (advmod_right_main) or (prep_after_verb) or (obj):
                # If adverb after verb exist ==> Subject + aux (optional) + not (optional) + adv (optional) + verb + adv + preposition phrase (optional)
                if advmod_right_main:
                    # Concatenate components into: adv (optional) + verb
                    ability = cross_product_str(pre_adv, head.text)
                    # Concatenate components into: not (optional) + adv (optional) + verb
                    ability = cross_product_str(neg, ability)
                    # Concatenate components into: not (optional) + adv (optional) + verb + adv
                    ability = cross_product_str(ability, post_adv)
        
                    # If aux exist, add the text into ability_text
                    if aux:
                        # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adv
                        ability = cross_product_str(aux.text, ability)

                    # If preposition phrase after adverb exist.
                    ######## EDIT HERE ################
                    prep_after_advmod = crawling_after_token_prep_phrase(advmod_right_main)
                    if prep_after_advmod:
                        # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adv + preposition phrase (optional)
                        ability = cross_product_str(ability, prep_after_advmod)
                    
                    # EXPECTED PATTERN: Subject + aux (optional) + not (optional) + adv (optional) + verb + adv + prepositional phrase (optional)
                    abilities += ability

                # If prep after verb ==> Subject + aux (optional) + not (optional) + adv (optional) + verb + preposition phrase
                if prep_after_verb:
                    # Concatenate components into: adv (optional) + verb
                    ability = cross_product_str(pre_adv, head.text)
                    # Concatenate components into: not (optional) + adv (optional) + verb
                    ability = cross_product_str(neg, ability)
                    # If auxiliary token exist
                    if aux:
                        # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                        ability = cross_product_str(aux.text, ability)
    
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + preposition phrase
                    ability = cross_product_str(ability, prep_after_verb)
    
                    # EXPECTED PATTERN: Subject + aux (optional) + not (optional) + adv (optional) + verb + preposition phrase
                    abilities += ability

                # If direct object exist ==> Subject + aux (optional) + not (optional) + adv (optional) + verb + adj (optional) + Direct object
                if obj:
                    # If the neglection do not appear as children's verb
                    if not neg:
                        neg = get_neglect(obj)
                    # Get pre adjective modifier direct object and convert in into string
                    pre_adj = ' '.join(extract_pre_adj(obj))
                    
                    # Concatenate components into: adv (optional) + verb
                    ability = cross_product_str(pre_adv, head.text)
                    # Concatenate components into: not (optional) + adv (optional) + verb
                    ability = cross_product_str(neg, ability)
                    # If auxiliary token exist
                    if aux:
                        # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                        ability = cross_product_str(aux.text, ability)
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adj (optional)
                    ability = cross_product_str(ability, pre_adj)
                    # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb + adj (optional) + Direct object
                    ability = cross_product_str(ability, obj.text)
                    
                    # EXPECTED PATTERN: Subject + aux (optional) + not (optional) + adv (optional) + verb + adj (optional) + Direct object
                    abilities += ability

                    # If the object has conjunct
                    obj_conjunct = extract_conj(obj)
                    if len(obj_conjunct) > 0:
                        # Concatenate components into: Verb (Have) + Conjunct
                        ability = cross_product_str(head.text, obj_conjunct)
                        # Concatenate components into: not (optional) + Verb (Have) + Conjunct
                        ability = cross_product_str(neg, ability)
    
                        # Add the ability into abilities
                        abilities += ability
                        # EXPECTED PATTERN: Subject + not (optional) + Verb (Have) + Direct object

            else:
                # If do not contain any of that, but intransitive verb ==> Subject + aux (optional) + not (optional) + adv (optional) + verb
                if int_rate > 0.5:
                    # Concatenate components into: adv (optional) + verb
                    ability = cross_product_str(pre_adv, head.text)
                    # Concatenate components into: not (optional) + adv (optional) + verb
                    ability = cross_product_str(neg, ability)
        
                    # If aux exist, add the text into ability_text
                    if aux:
                        # Concatenate components into: aux (optional) + not (optional) + adv (optional) + verb
                        ability = cross_product_str(aux.text, ability)

                    # EXPECTED PATTERN: aux (optional) + not (optional) + adv (optional) + verb
                    abilities += ability

    # If head is aux
    elif head.pos_ == 'AUX':
        # Get the token
        # NOTE: if 'AUX' is root, only have one adjective with dependency acomp.
        adj_token = get_token_dep(head, dep='acomp')
        noun_token = get_token_pos_right(head, pos=['NOUN', 'PROPN'])

        if adj_token:
            # Get Preposition after adjective
            prep_after_adj = ' '.join(crawling_after_token_prep_phrase(adj_token))

            # Concatenate components into: aux + not (optional)
            ability = cross_product_str(head.text, neg)
            # Concatenate components into: aux + not (optional) + adj
            ability = cross_product_str(ability, adj_token.text)
            # Concatenate components into: aux + not (optional) + adj + preposition phrase (optional)
            ability = cross_product_str(ability, prep_after_adj)

            # EXPECTED PATTERN: Subject + aux + not (optional) + adj
            abilities += ability

            # Get all conjunct (except adjective with dependency acomp)
            if neg:
                adj_conjunct = extract_conj(adj_token)
            else:
                # If neglect do not come at first, then check neglection in front each conjunct.
                adj_conjunct = extract_conj(adj_token, neglect=True)

            if len(adj_conjunct) > 0:
                # Concatenate components into: aux + not (optional)
                ability = cross_product_str(head.text, neg)
                # Concatenate components into: aux + not (optional) + adj
                ability = cross_product_str(ability, adj_conjunct)
                # EXPECTED PATTERN: Subject + aux + not (optional) + adj
                abilities += ability

        elif noun_token:
            # Get Pre-modifier adjective
            pre_adj_noun = ' '.join(extract_pre_adj(noun_token))
            # Concatenate components into: aux + not (optional)
            ability = cross_product_str(head.text, neg)
            # Concatenate components into: aux + not (optional) + pre-modifier adjective (optional)
            ability = cross_product_str(ability, pre_adj_noun)
            # Concatenate components into: aux + not (optional) + pre-modifier adjective (optional) + noun
            ability = cross_product_str(ability, noun_token.text)

            # EXPECTED PATTERN: Subject + aux + not (optional) + pre-modifier adjective (optional) + noun
            abilities += ability

            # Get all conjunct
            if neg:
                noun_conjunct = extract_conj(noun_token)
            else:
                # If neglect do not come at first, then check neglection in front each conjunct.
                noun_conjunct = extract_conj(noun_token, neglect=True)

            if len(noun_conjunct) > 0:
                # Concatenate components into: aux + not (optional)
                ability = cross_product_str(head.text, neg)
                # Concatenate components into: aux + not (optional) + noun
                ability = cross_product_str(ability, noun_conjunct)
                # EXPECTED PATTERN: Subject + aux + not (optional) + noun
                abilities += ability
    return abilities